# Read and process AIS data

- Read AIS (ship location) data from the Danish costguard
- See ship traffic for a day
- Locate any ships sailing into the area between the offshore turbines

In [ ]:
# Standard library
from pathlib import Path
import zipfile

# Data analysis
import pandas as pd

# Geospatial libraries
import geopandas as gpd
import pydeck as pdk


In [ ]:
if not Path("aisdk-2023-01.zip").exists():
    raise Exception("Unable to find AIS data. Please download it from http://web.ais.dk/aisdata/")

In [ ]:
# Read all ship location information and track first and last position of each day
with zipfile.ZipFile("aisdk-2023-01.zip") as z:
    # Load only the first day
    with z.open("aisdk-2023-01-01.csv") as f:
        ships = pd.read_csv(f)

In [ ]:
# Remove ships that don't transmit a callsign
ships.dropna(subset=('Callsign'), inplace=True)

# Clean up raw data
ships.sort_values(by=['Callsign', '# Timestamp'], inplace=True)

In [ ]:
# Join all GPS points to a single line for all ships
ship_track = []
for g in ships.groupby('Callsign')[['Longitude', 'Latitude']]:
    path = g[1]
    path = path.loc[path['Latitude'] < 70, :]
    path = path.to_numpy()[::10].tolist()
    if len(path) > 10 and g[0] != "0":
        ship_track.append({"Callsign": str(g[0]), 'path': path})
    
df_ship_track = pd.DataFrame.from_dict(ship_track)

In [ ]:
# Load wind turbine offshore locations
json = pd.read_json('../../NoCode/data/dk_offshore_sites.geojson')
sites = pd.DataFrame()

# Parse the geometry out in Pandas
sites["coordinates"] = json["features"].apply(lambda row: row["geometry"]["coordinates"])

In [ ]:
# Make map
view_state = pdk.ViewState(latitude=55.782556, longitude=11.3484867, zoom=5)

# Draw ship routes as paths
layer = pdk.Layer(
    type="PathLayer",
    data=df_ship_track,
    pickable=True,
    width_scale=20,
    get_color=(255,255,255),
    width_min_pixels=2,
    get_path="path",
    get_width=5,
)

# Draw turbine locations as polygons
layer_turbines = pdk.Layer(
    type="PolygonLayer",
    data=sites,
    pickable=True,
    get_fill_color=[178,34,34],
    get_line_color=[255, 255, 255],
    get_polygon="coordinates",
    opacity=0.8,
    auto_highlight=True,
)

r = pdk.Deck(layers=[layer, layer_turbines], initial_view_state=view_state, tooltip={"text": "{Callsign}"})
r.to_html("path_layer.html")

## Extract only ships that sail in between the turbines and draw their port of origin and destination

In [ ]:
# Load both ship locations and wind locations as native spatial types
geo_sites = gpd.read_file("../../NoCode/data/dk_offshore_sites.geojson")
geo_ships = gpd.GeoDataFrame(ships, geometry=gpd.points_from_xy(ships.Longitude, ships.Latitude), crs="EPSG:4326")

In [ ]:
# Find all ship locations within the wind parks
points_within_site = geo_sites.sjoin(geo_ships, how='left').dropna(subset='MMSI')

In [ ]:
# Make map
view_state = pdk.ViewState(latitude=55.782556, longitude=11.3484867, zoom=5)

# Draw ship routes as paths
layers = [
    pdk.Layer(
        "ScatterplotLayer",
        data=points_within_site,
        get_position=['Longitude', 'Latitude'],
        auto_highlight=True,
        get_radius=300,          # Radius is given in meters
        get_fill_color=[180, 0, 200, 140],  # Set an RGBA value for fill
        pickable=True
    )
]

r = pdk.Deck(layers=layers, initial_view_state=view_state, tooltip={"text": "{Callsign}"})
r.to_html("path_layer.html")